In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
path = os.path.join(os.getcwd(), '../raw_data')

data_prep = pd.read_csv(os.path.join(path, 'x_train_preprocessed.csv'))
data_prep

,Unnamed: 0,summary,positives,negatives,advice_to_mgmt,review,summary_length,postives_length,negatives_length,advice_length,...,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,0,good job considering retail,apple good company period pay average working ...,job movement lateral extremely difficult get p...,NaN,good job considering retail apple good company...,31,118,127,0,...,0.600000,0.700000,0.666667,0.516667,1.000000,-0.500000,0.000000,0.000000,0.720000,0.350000
1,1,warning potential new hire,good pay smart coworkers good insurance,hr protect company employee careful restricted...,many employment contract state restricted stoc...,warning potential new hire good pay smart cowo...,31,41,132,1509,...,0.727273,0.068182,0.614286,0.538095,0.625000,0.075000,0.499852,0.120897,0.536005,0.157255
2,2,demanding job get work brilliant mind industry,intelligent co worker encouraged innovate good...,promote work life balance word employee get re...,employee get burned soon life outside work goi...,demanding job get work brilliant mind industry...,87,89,164,150,...,1.000000,0.900000,0.833333,0.783333,0.000000,0.000000,0.368750,-0.156250,0.706250,0.489583
3,3,deliver get fired,pay competitive look good resume move around i...,work life balance hostile environment reward b...,people disposable,deliver get fired pay competitive look good re...,21,189,306,26,...,0.000000,0.000000,0.460714,0.403571,0.492063,0.031746,0.000000,0.000000,0.474150,0.244218
4,4,good salary perk benefit,good salary perk benefit,hectic schedule u ill learning cutting technology,NaN,good salary perk benefit good salary perk bene...,34,34,57,0,...,0.600000,0.700000,0.600000,0.700000,0.950000,-0.550000,0.000000,0.000000,0.775000,0.075000
5,5,happy year working amazon,great people challenging project fast paced en...,hard get often turn away good people left clea...,great hiring need get better,happy year working amazon great people challen...,45,59,108,50,...,1.000000,0.800000,0.783333,0.500000,0.381250,0.127083,0.625000,0.650000,0.612500,0.410833
6,6,one hopefully soon remembered,great experience advancement understanding uti...,always enough time meet deadline,keep positive attitude try cooperate much poss...,one hopefully soon remembered great experience...,35,121,40,80,...,0.000000,0.000000,0.575000,0.450000,0.500000,0.000000,0.772727,0.113636,0.639091,0.225455
7,7,hardwork peanut,fellow team member great work,limited opportunity move within company,try treat team member like robot like human,hardwork peanut fellow team member great work ...,20,44,50,67,...,0.000000,0.000000,0.750000,0.800000,0.142857,-0.071429,0.100000,0.000000,0.330952,0.242857
8,8,software developer,really nothing share v,everything terrible v,NaN,software developer really nothing share v ever...,18,26,28,0,...,0.000000,0.000000,0.200000,0.200000,1.000000,-1.000000,0.000000,0.000000,0.600000,-0.400000
9,9,everyone high stress company growing pain,world class benefit smart coworkers food galor...,work life balance unrealistic recruiting expec...,stop micromanaging sourcers current recruiting...,everyone high stress company growing pain worl...,56,60,235,238,...,0.540000,0.160000,0.421429,0.257143,0.644444,-0.300000,0.594444,-0.133333,0.566614,-0.069524


In [7]:
positives = data_prep.positives.dropna()
positives = positives[:100]
positives.shape

(100,)

In [9]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [10]:
vectorizer = TfidfVectorizer(max_df=0.8, ngram_range=(3,3)).fit(positives) # words that appear most (top 20%) left out

data_vectorized = vectorizer.transform(positives)

lda_model = LatentDirichletAllocation(n_components=3).fit(data_vectorized)

print_topics(lda_model, vectorizer)

Topic 0:
[('salary perk benefit', 1.0658015118991773), ('team work international', 1.0364846389331612), ('work international echo', 1.0364846389331608), ('good salary perk', 1.0062545741311564), ('technology impact scale', 0.9074235199556544), ('impact scale money', 0.9074235199556542), ('people technology impact', 0.9074235199556542), ('right benefit seasonal', 0.9074232960918351), ('benefit seasonal employee', 0.9074232960918351), ('pay right benefit', 0.9074232960918351)]
Topic 1:
[('great place work', 1.3282634669354927), ('career potential driven', 1.0368324623273668), ('great career potential', 1.0368324623273666), ('people work hard', 0.9234568330396384), ('work hard push', 0.9234568330396384), ('passion enriching life', 0.9077119939002121), ('exploring passion enriching', 0.9077119939002121), ('innovative exploring passion', 0.9077119939002121), ('co worker miserable', 0.9077118452076309), ('pay co worker', 0.9077118452076309)]
Topic 2:
[('really nothing share', 1.3281657106150

In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import string
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/theresasporn/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/theresasporn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
tokenized = []
for sentence in positives:
    tokenized.append(word_tokenize(sentence))
tokenized   

[['apple',
  'good',
  'company',
  'period',
  'pay',
  'average',
  'working',
  'retail',
  'store',
  'benefit',
  'excellent'],
 ['good', 'pay', 'smart', 'coworkers', 'good', 'insurance'],
 ['intelligent',
  'co',
  'worker',
  'encouraged',
  'innovate',
  'good',
  'benefit',
  'beautiful',
  'campus',
  'location'],
 ['pay',
  'competitive',
  'look',
  'good',
  'resume',
  'move',
  'around',
  'internally',
  'year',
  'lot',
  'smart',
  'people',
  'work',
  'amazon',
  'dog',
  'allowed',
  'good',
  'engineer',
  'school'],
 ['good', 'salary', 'perk', 'benefit'],
 ['great', 'people', 'challenging', 'project', 'fast', 'paced', 'environment'],
 ['great',
  'experience',
  'advancement',
  'understanding',
  'utilize',
  'time',
  'manage',
  'creation',
  'complete',
  'task'],
 ['fellow', 'team', 'member', 'great', 'work'],
 ['really', 'nothing', 'share', 'v'],
 ['world',
  'class',
  'benefit',
  'smart',
  'coworkers',
  'food',
  'galore',
  'fun',
  'party'],
 ['peopl

In [18]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [2]:
from data import get_data


ImportError: cannot import name 'get_data' from 'data' (unknown location)